In [1]:
import pandas as pd
from scipy.spatial.distance import cosine
from pandas.io.json import json_normalize #package for flattening json in pandas df
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas_ml as pdml

In [2]:
#specify the file path
filename="D:/shopper reco/training_mixpanel.txt"


In [6]:
#open the file and load the json
with open(filename) as f:
    d = json.load(f)

    

In [7]:
#Normalise the json to generate the required columns
data = json_normalize(d)
data.head(3)

,event,properties.country,properties.customer_id,properties.description,properties.invoice_date,properties.invoice_no,properties.product_id,properties.quantity,properties.unit_price
0,Purchased Product,United Kingdom,17850,WHITE HANGING HEART T-LIGHT HOLDER,12/1/2010 8:26,536365,85123A,6,2.55
1,Purchased Product,United Kingdom,17850,WHITE METAL LANTERN,12/1/2010 8:26,536365,71053,6,3.39
2,Purchased Product,United Kingdom,17850,SET 7 BABUSHKA NESTING BOXES,12/1/2010 8:26,536365,22752,2,7.65


In [10]:
data.columns=['event','country','customer_id','description','invoice_date','invoice_no','product_id','quantity','unit_price']


In [11]:
#calcualte the total sales value for each event
data['sales']=data['quantity']*data['unit_price']

In [49]:
#calculate the aggregate sales  for each product customer and country and the number of products bought by each customer
grpdata=data.groupby(['customer_id','product_id','country'])['sales'].agg(['mean', 'count'])

In [50]:
grpdata=grpdata.reset_index()

grpdata

,customer_id,product_id,country,mean,count
0,12346,23166,United Kingdom,0.000000,2
1,12347,16008,Iceland,6.000000,1
2,12347,17021,Iceland,10.800000,1
3,12347,20665,Iceland,17.700000,1
4,12347,20719,Iceland,8.500000,4
5,12347,20780,Iceland,55.800000,1
6,12347,20782,Iceland,32.940000,1
7,12347,20966,Iceland,12.500000,1
8,12347,21035,Iceland,17.700000,1
9,12347,21041,Iceland,17.700000,2


In this use case we have to predict the likelihood of customer going to buy a product. Since the current data is about purchases already made, we have to predict the propensity of the next purchase of the same product by a given customer. We cannot predict the likehood of purchase which a customer has never purchased based on the dataset provided. So this weill be a use case of predicting the propensity of next purchase

Next we generate a target variable which will tell us whether a customer is likely to purchase again or not. So if a customer has bought more than 1 product ,he is like to buy another product again ,So we can use this to generate a target variable where if a person has a count of purchase more than 1 he is like to purchase that product again otherwise he will not purchase again ,Our likelihood to buy score will be based on this target vaiable. We will use the mean sales, the count of sales and the country as the dependent variable

In [51]:
grpdata['target'] = np.where(grpdata['count']>1, 1, 0)
target=grpdata['target'] 
features=grpdata.iloc[:,3:5]
#Lets use one hot encoding to convert categorical variables to one hot
from sklearn.preprocessing import LabelBinarizer
# Create LabelBinzarizer object
one_hot = LabelBinarizer()
country=pd.get_dummies(grpdata['country'])
newfeatures=pd.concat([features,country],axis=1)
newfeatures

,mean,count,Australia,Austria,Bahrain,Belgium,Brazil,Canada,Channel Islands,Cyprus,...,RSA,Saudi Arabia,Singapore,Spain,Sweden,Switzerland,USA,United Arab Emirates,United Kingdom,Unspecified
0,0.000000,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,6.000000,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10.800000,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,17.700000,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8.500000,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,55.800000,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,32.940000,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,12.500000,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,17.700000,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,17.700000,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#Data needs to be balanced since we have a high proportion of 0's we are balancing the data to have equal proportion of0's and 1.We will Use SMOTE to balance the data

In [82]:
df = pdml.ModelFrame(newfeatures,target=target)
df
df.target.value_counts()
sampler = df.imbalance.over_sampling.SMOTE()
sampled = df.fit_sample(sampler)


Use the SMOTE sampled dataset to seperate out the target variables and the indpendent variable

In [83]:
sampled
sampledframe=pd.DataFrame(sampled)
features=sampledframe.iloc[:,1:40]
target=sampledframe.iloc[:,0]
features

,mean,count,Australia,Austria,Bahrain,Belgium,Brazil,Canada,Channel Islands,Cyprus,...,RSA,Saudi Arabia,Singapore,Spain,Sweden,Switzerland,USA,United Arab Emirates,United Kingdom,Unspecified
0,0.000000,2.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,6.000000,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10.800000,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,17.700000,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8.500000,4.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,55.800000,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,32.940000,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,12.500000,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,17.700000,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,17.700000,2.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0



#Now we have a binary clasiifier for each product and customer id combination. We will use classification algorithms to generate the likelihood of customer buying the product again. 



Lets first use kneighbours

In [57]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.5)
neigh = KNeighborsRegressor(n_neighbors=3)
# classifications
neighfit=neigh.fit(X_train,y_train) 


predictions=neighfit.predict(X_test)
y_test.values

array([0, 1, 0, ..., 0, 0, 0])

In [58]:
predictions=np.array(predictions)
predictions=predictions.astype(int)

In [59]:
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(predictions,y_test.values)             
metrics.auc(fpr, tpr)

0.99881240269544991

Lets try logistic regression

In [86]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.5)
logistic = linear_model.LogisticRegression()
# classifications
logfit=logistic.fit(X_train,y_train) 

logpredictions=logfit.predict(X_test)
y_test.values

array([0, 1, 0, ..., 1, 1, 0])

In [87]:
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(logpredictions,y_test.values)             
metrics.auc(fpr, tpr)

1.0

Lets try random forest

In [88]:
from sklearn.ensemble import RandomForestClassifier
rffit=RandomForestClassifier(max_depth=2, random_state=0)
# classifications
rffit=rffit.fit(X_train,y_train) 

rfpredictions=rffit.predict(X_test)
y_test.values

array([0, 1, 0, ..., 1, 1, 0])

In [89]:
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(rfpredictions,y_test.values)             
metrics.auc(fpr, tpr)

0.99998737979252372

In [91]:
newfeatures

,mean,count,Australia,Austria,Bahrain,Belgium,Brazil,Canada,Channel Islands,Cyprus,...,RSA,Saudi Arabia,Singapore,Spain,Sweden,Switzerland,USA,United Arab Emirates,United Kingdom,Unspecified
0,0.000000,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,6.000000,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10.800000,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,17.700000,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8.500000,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,55.800000,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,32.940000,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,12.500000,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,17.700000,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,17.700000,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#We are getting good area under the curve for both Logistic regression and Random forest. Since random forest is an ensemble method we will use random forest for final predictions

In [93]:

finalpredictions=rffit.predict_proba(newfeatures)
finalpredictions=pd.DataFrame(finalpredictions[:,1])
finalpredictions=pd.concat([grpdata.iloc[:,0:3],finalpredictions],axis=1)
finalpredictions

,customer_id,product_id,country,0
0,12346,23166,United Kingdom,0.589258
1,12347,16008,Iceland,0.389258
2,12347,17021,Iceland,0.389258
3,12347,20665,Iceland,0.389258
4,12347,20719,Iceland,0.589258
5,12347,20780,Iceland,0.369559
6,12347,20782,Iceland,0.369559
7,12347,20966,Iceland,0.389258
8,12347,21035,Iceland,0.389258
9,12347,21041,Iceland,0.589258


In [94]:
finalpredictions.columns=['customer_id','product_id','country','repurchasepropensity']
finalpredictions.to_csv('D:/shopper reco/finalrecommendations.csv')

In [43]:
finalpredictions.values


array([[12346, '23166', 'United Kingdom', 0.5922535938979155],
       [12347, '16008', 'Iceland', 0.3922535938979156],
       [12347, '17021', 'Iceland', 0.3922535938979156],
       ..., 
       [18287, '84920', 'United Kingdom', 0.3922535938979156],
       [18287, '85039A', 'United Kingdom', 0.6715855566802963],
       [18287, '85040A', 'United Kingdom', 0.6715855566802963]], dtype=object)